In [ ]:
import sys
import os
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import pandas as pd
import xarray as xr
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from lesview import *
from sbl_bbl import *

In [ ]:
g = 9.81
H = 30
u10 = 8
N2 = 1.962e-4
bstar = N2 * H
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
lat = 45.
tau = rhoa/rhoo*cd*u10*u10
ustar = np.sqrt(tau)
amplitude = 1.0
wavelength = 60
wavenumber = 2.*np.pi/wavelength
frequency = np.sqrt(g*wavenumber*np.tanh(wavenumber*H))
us0 = amplitude**2*wavenumber
la = np.sqrt(ustar/us0) 
print('La = {:6.3f}'.format(la))
Ti = inertial_period(lat)
print('Ti = {:6.3f}'.format(Ti))

In [ ]:
reslist = ['n128l72', 'n256l144', 'n512l288', 'n128l144', 'n256l72', 'n256l288']

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_v2'
figpath  = 'overview_{:s}'.format(casename)
os.makedirs(figpath, exist_ok=True)

In [ ]:
ds1_pfls = {}
ds2_pfls = {}
for res in reslist:
    datapath = os.path.join(os.path.pardir, 'tests', '{:s}_{:s}'.format(casename, res))
    filepath = os.path.join(datapath, 'averages.jld2')
    ds1_pfls[res] = OceananigansDataProfile(filepath=filepath).dataset
    datapath = os.path.join(os.path.pardir, 'tests', '{:s}_rf_{:s}'.format(casename, res))
    filepath = os.path.join(datapath, 'averages.jld2')
    ds2_pfls[res] = OceananigansDataProfile(filepath=filepath).dataset

In [ ]:
def get_das(ds):
    das = dict(
        NN = ds.data_vars['b'].differentiate(coord='z')/N2,
        SS = (ds.data_vars['u'].differentiate(coord='z')**2+ds.data_vars['v'].differentiate(coord='z')**2)/N2,
        # Ri = ds.data_vars['b'].differentiate(coord='z')/(ds.data_vars['u'].differentiate(coord='z')**2+ds.data_vars['v'].differentiate(coord='z')**2),
        Ri = (ds.data_vars['b'].differentiate(coord='z')-0.25*(ds.data_vars['u'].differentiate(coord='z')**2+ds.data_vars['v'].differentiate(coord='z')**2))/N2,
        wb = (ds.data_vars['wb']+ds.data_vars['wbsb'])/ustar/bstar*1e3,
        wu = (ds.data_vars['wu']+ds.data_vars['wusb'])/ustar**2,
        wv = (ds.data_vars['wv']+ds.data_vars['wvsb'])/ustar**2,
    )
    return das

In [ ]:
edges1 = {}
edges2 = {}
for res in reslist:
    edges1[res], edges2[res] = [get_edges(nondim_da(ds.data_vars['b'].differentiate(coord='z')/N2, H=H, Tf=Ti)) for ds in [ds1_pfls[res], ds2_pfls[res]]]

In [ ]:
colors = dict(
    n128l72   = 'steelblue',
    n256l72   = 'mediumslateblue',
    n128l144  = 'skyblue',
    n256l144  = 'k',
    n256l288  = 'lightcoral',
    n512l288  = 'firebrick',
)
labels = dict(
    n128l72   = '$128^2\\times72$',
    n256l72   = '$256^2\\times72$',
    n128l144  = '$128^2\\times144$',
    n256l144  = '$256^2\\times144$',
    n256l288  = '$256^2\\times288$',
    n512l288  = '$512^2\\times288$',
)
linewidths = dict(
    n128l72   = 1.5,
    n256l72   = 0.75,
    n128l144  = 0.75,
    n256l144  = 1.5,
    n256l288  = 0.75,
    n512l288  = 1.5,
)
zorders = dict(
    n128l72   = 3,
    n256l72   = 1,
    n128l144  = 1,
    n256l144  = 3,
    n256l288  = 1,
    n512l288  = 3,
)
abc = ['ad', 'be', 'cf']
tags = ['Aligned', 'Opposite']
tagy = ['bottom', 'top']
texty = {'top': 0.92, 'bottom': 0.08}
U0 = [0.25, -0.25]
xlabels = ['$(\overline{u}-U_0)/u_*$', '$\overline{w^\prime u^\prime}/u_*^2$']
fig, axarr = plt.subplots(2, 3, sharey='row', gridspec_kw={'width_ratios': [3, 1, 1]})
fig.set_size_inches(8,5)
dss_pfls = [ds1_pfls, ds2_pfls]
for k, edges in enumerate([edges1, edges2]):
    ax = axarr[k,0]
    for res in reslist:
        edges[res][0].rolling(time=5, center=True).mean().plot(ax=ax, color=colors[res], linewidth=linewidths[res], linestyle='-', zorder=zorders[res], label=labels[res])
        edges[res][1].rolling(time=5, center=True).mean().plot(ax=ax, color=colors[res], linewidth=linewidths[res], linestyle='-', zorder=zorders[res])
    ax.set_ylabel('$z/H$')
    ax.set_ylim([-1, 0])
    ax.set_xlim([0, 16])
    ax.text(0.05, texty['top'], '({:s})'.format(abc[0][k]), transform=ax.transAxes, va='top', ha='left')
    ax.text(0.95, texty[tagy[k]], tags[k], transform=ax.transAxes, va=tagy[k], ha='right')
    if k == 0:
        ax.legend(loc='upper right', ncol=2, fontsize=9)
        ax.set_xlabel('')
    else:
        ax.set_xlabel('$t/T_f$')
        
    for res in reslist:
        time_merge = edges[res][0].dropna(dim='time').time[-1]
        tslice1 = slice(time_merge - 2, time_merge - 1)
        tslice2 = slice(time_merge + 4, time_merge + 5)
        ds = dss_pfls[k][res]
        da = nondim_da((ds.data_vars['u']-U0[k])/ustar, H=H, Tf=Ti)
        da.sel(time=tslice1).mean(dim='time').plot(ax=axarr[k,1], y=da.dims[0], color=colors[res], linewidth=linewidths[res], linestyle='--', zorder=zorders[res])
        da.sel(time=tslice2).mean(dim='time').plot(ax=axarr[k,1], y=da.dims[0], color=colors[res], linewidth=linewidths[res], linestyle='-', zorder=zorders[res])
        da = nondim_da((ds.data_vars['wu']+ds.data_vars['wusb'])/ustar**2, H=H, Tf=Ti)
        da.sel(time=tslice1).mean(dim='time').plot(ax=axarr[k,2], y=da.dims[0], color=colors[res], linewidth=linewidths[res], linestyle='--', zorder=zorders[res])
        da.sel(time=tslice2).mean(dim='time').plot(ax=axarr[k,2], y=da.dims[0], color=colors[res], linewidth=linewidths[res], linestyle='-', zorder=zorders[res])

    for i in np.arange(2):
        ax = axarr[k,i+1]
        ax.set_ylabel('')
        ax.text(0.9, texty['top'], '({:s})'.format(abc[i+1][k]), transform=ax.transAxes, va='top', ha='right')
        if k == 0:
            ax.set_xlabel('')
        else:
            ax.set_xlabel(xlabels[i])
        ax.axvline(x=0, linewidth=0.75, color='k', zorder=0)

line_kwargs1 = dict(linestyle='--', linewidth=1)
line_kwargs2 = dict(linestyle='-', linewidth=1)
l1, = plt.plot(np.nan,np.nan,'k',**line_kwargs1)
l2, = plt.plot(np.nan,np.nan,'k',**line_kwargs2)
axarr[1,1].legend([l1,l2],['T1','T2'], loc='center right', fontsize=9)

plt.tight_layout()
plt.subplots_adjust(wspace=0.12)
figname = os.path.join(figpath, 'resolution-sensitivity_v2')
fig.savefig(figname, dpi = 300, facecolor='w')